The solution for the question sets can be accesed through the following link
  * [Flat Iron Data Exercise](http://nbviewer.jupyter.org/github/sushmit86/Data_science_experiments/blob/master/Flat_iron/flat_iron_exersices_sushmit_roy.ipynb)

For a better UI experience and visulaization it is recomended to use the above links  
Section "*Loading Data to My-sql local instance*" can be skipped and one can jump to "*Solutions to Exercise*" section

In [1]:
# import Python libarary 
import pymysql
import pandas as pd
from bokeh.layouts import gridplot
#from bokeh.plotting import figure, output_notebook, show,Histogram
from bokeh.charts import Histogram,BoxPlot, output_notebook, show
import numpy as np
output_notebook()

Loading BokehJS ...

# Loading Data to My-sql local instance

Creating a new schema **flat_iron in a  mysql(5.7.13)  local database** and creating tables for Patient_Diagnosis.csv &
Patient_Treatment.csv 

In [2]:
# Creating a connection 
# note the flat_iron schema was manually created 
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='password', db='flat_iron')
c = conn.cursor()

In [3]:
# creating patient_diagnosis table
create_patient_diagnosis = """
CREATE TABLE if not exists flat_iron.patient_diagnosis (
   Patient_ID INT NOT NULL,
   Diagnosis_Date DATE NOT NULL,
   Diagnosis_Code DECIMAL(7,2) NOT NULL,
   Diagnosis VARCHAR(45) NULL
)
"""
c.execute(create_patient_diagnosis)

# loading the data from Patient_Diagnosis.csv
loading_patient_diagnosis = r"""
LOAD DATA 
INFILE 
'/Users/sushmitroy/Data_science_experiments/Flat_iron/Patient_Diagnosis.csv'
INTO TABLE  flat_iron.patient_diagnosis
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
"""
c.execute(loading_patient_diagnosis)
# commit
conn.commit()

In [4]:
# creating patient_treatment.csv
create_patient_diagnosis = """
CREATE TABLE if not exists flat_iron.patient_treatment (
   Patient_ID INT NOT NULL,
   Treatment_Date DATE NOT NULL,
   Drug_Code INT NOT NULL
)
"""
c.execute(create_patient_diagnosis)

# loading the data from Patient_Treatment.csv
loading_patient_diagnosis = r"""
LOAD DATA 
INFILE 
'/Users/sushmitroy/Data_science_experiments/Flat_iron/Patient_Treatment.csv'
INTO TABLE  flat_iron.patient_treatment
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
"""
c.execute(loading_patient_diagnosis)
# commit
conn.commit()

# Solutions to Exercise

Exercise
1.  How many patients does the clinic have for each cancer diagnosis?

In [5]:
sql_exercise_1 = """
select 
Diagnosis,
count(distinct Patient_ID) as count
from flat_iron.patient_diagnosis
group by 1
order by 2 desc
"""
df_exercise_1 = pd.read_sql(sql_exercise_1,conn)
df_exercise_1

Diagnosis  count
0  Breast Cancer     26
1   Colon Cancer     12

Solution  
1.Count distinct Patient_ID in each diagnosis group. Results have been tabulated above.  
  Time Spent : 2 minutes

Exercise  
2 . How long after being diagnosed does it take for a patient to start treatment?  Do any patients start treatment before being diagnosed?

In [6]:
sql_exercise_2 = """
select 
t1.Patient_ID,
DATEDIFF(min_date,diag_date) as diff
from
(select 
Patient_ID,
min(Diagnosis_Date) as diag_date
from
flat_iron.patient_diagnosis
group by 1 ) t1
inner join

(SELECT Patient_ID,
min(Treatment_Date)  as min_date
from
flat_iron.patient_treatment
group by 1) t2
on t1.Patient_ID = t2.Patient_ID
order by 1 
"""
df_exercise_2 = pd.read_sql(sql_exercise_2,conn)
bins = np.arange(-60,80,20)
hist = Histogram(df_exercise_2, values='diff',
                 title="Time to Start Treatment",bins = list(bins),xlabel= 'Time in days', ylabel = 'count')
show(hist)



In [7]:
# start treatment before being diagnosed
df_exercise_2[df_exercise_2['diff'] < 0]

Patient_ID  diff
11        4374   -47
17        6877   -21

In [8]:
df_exercise_2[df_exercise_2['diff'] > 0].describe()['diff']

count    25.000000
mean     31.760000
std      15.658544
min       2.000000
25%      17.000000
50%      32.000000
75%      43.000000
max      70.000000
Name: diff, dtype: float64

Solution  
Following assumtions were made to answer this question
  * The minimum (or first in ascending order) date in patient_diagnosis data represents the start date of diagnosis
  * The minimum (or first in ascending order) date in patient_treatment data represents the start date of treatment
  * If there is no data for a Patient_ID in patient_diagnosis but not in patient_treatment we assume that treatment has not started for that particular patient. (Hence the inner join)

```We can see from the histogram there are two Patient_ID ( i.e 4374,6877) have treatment start date before diagnosis date, we assume this is due to data anamoly and we remove them to answer "How long after being diagnosed does it take for a patient to start treatment". 
Hence we can see from the above results that the average time for a patient to start diagnosis is 32 (31.76 ) days and there is a 75% probability to have the diagnosis started before 43 days``` 

Time spent : 15 Minutes


In [9]:
sql_exercise_3 = """
SELECT 
Patient_ID,
Drug_Code,
DATEDIFF(max(Treatment_Date), min(Treatment_Date)) as len_treatment
from
flat_iron.patient_treatment
group by 1,2
order by 2 desc
"""
df_exercise_3 = pd.read_sql(sql_exercise_3,conn)

In [10]:
### Box plot 
p = BoxPlot(df_exercise_3, values='len_treatment', label='Drug_Code',
            title="Boxplot of length of treatment between Drug 201 and Drug 202",color='red')
show(p)

## Print mean of each drug group
print('Mean of Each Group')
print(df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.mean))


### do a Null hypothesis test using permutation test

observed1 = df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.median)[201] 
observed2 = df_exercise_3.pivot_table( columns= 'Drug_Code' , values= 'len_treatment', aggfunc= np.median)[202] 
observed = observed1 - observed2

result = pd.Series(np.zeros(10**4 -1))
def permute_test(d  = 0):
    n = len(df_exercise_3)
    c = len(df_exercise_3[df_exercise_3['Drug_Code']==201])             
    index=np.random.choice(np.arange(0,n,1), size = c, replace = False)
    not_index = np.setdiff1d(np.arange(0,n,1),index , assume_unique = True)
    return df_exercise_3['len_treatment'].iloc[index].median() - df_exercise_3['len_treatment'].iloc[not_index].median()
result=result.apply(permute_test)
# calculating P value
# calculating P value
p_value = ((result >= observed).sum() +1)/(10**4)
#### Print p value
print('\n p value is : ')
print(p_value)

Mean of Each Group
Drug_Code
201    72.000000
202    10.769231
Name: len_treatment, dtype: float64

 p value is : 
0.0002


Exercise  
3 . Is there a difference between Drug 201 and Drug 202 in terms of length of treatment?

Following assumptions were made to answer the question
  * The minimum (or first in ascending order) date in patient_treatment data represents the start date of treatment
  * The maximum (or last in ascending order) date in patient_treatment data represents the end date of treatment
  * length of treatment = End date - Start date 
  
We cleary notice from the Box plot that Drug 201 has higher length of treatment than Drug 202. Also the mean for Drug 201 is 72 days while for Drug 202 is 11(10.76) days.

 Permutation test with the following Hypothesis:
 * $H_0$ : $\mu_{201} = \mu_{202}$
 * $H_A$ : $\mu_{201} > \mu_{202}$
 
where $\mu_{201} \ and \  \mu_{202}$ are median for Drug 201 and Drug 202 respectively

```We do 10000 iteration and observe a p value = 0.002 , hence we can reject the null hypothesis at a 1% significance level and accept the alternative hypothesis```

Hence we can conclude that Drug 201 has higher length of treatment than Drug 202

Time spent : 30 minutes




In [11]:
# close the connection
conn.close()